# A ResNet implementation with PyTorch

source: 
* https://www.youtube.com/watch?v=DkNIBBBvcPs
* https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/ (yet to study)

In [1]:
import torch
import torch.nn as nn

# Setup tools
# python = 3.9.6
# conda install -c anaconda ipykernel
# python -m ipykernel install --user --name=ResNet
# pip install flake8 black isort mypy numpy pandas torch torchvision matplotlib jupyter scipy
# conda install cudatoolkit
# conda install -c pytorch torchvision cudatoolkit=10.1 pytorch

In [5]:
class identity_block(nn.Module):
    """
    Implementation of the identity block used in ResNet
    """
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(identity_block, self).__init__()
        self.expansion = 4 # Factor to expand the dimesion of channels
        self.layer_stack = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels*self.expansion),
            nn.ReLU(),
        )
        self.identity_downsample = identity_downsample
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x
        x = self.layer_stack(x)
        
        # Skip connection
        # Add shortcut value to main path, and pass it through a RELU activation 
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x += identity
        x = self.relu(x)
        
        return x

class ResNet(nn.Module): # [3, 4, 6, 3]
    """
    Implementation of ResNet
    """
    def __init__(self, identity_block, layers, image_channles, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channles, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # ResNet layers with skip connection
        self.layer1 = self._make_layer(identity_block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(identity_block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(identity_block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(identity_block, layers[3], out_channels=512, stride=2)
        
        self. avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*4, num_classes) # fully connectd
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
        
        
    def _make_layer(self, identity_block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []
        
        # Add skip connection
        if stride != 1 or self.in_channels != out_channels * 4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1, stride=stride), 
                                                nn.BatchNorm2d(out_channels*4))
        layers.append(identity_block(self.in_channels, out_channels, identity_downsample, stride)) # changes the number of channels
        self.in_channels = out_channels * 4 # 64*4 = 256
        
        for i in range(num_residual_blocks - 1):
            layers.append(identity_block(self.in_channels, out_channels)) # 256 -> 64, 64*4 (256) agian
            
        return nn.Sequential(*layers)
            
def ResNet50(img_channels=3, num_classes=1000):
    return ResNet(identity_block, [3, 4, 6, 3], img_channels, num_classes)

def ResNet101(img_channels=3, num_classes=1000):
    return ResNet(identity_block, [3, 4, 23, 3], img_channels, num_classes)

def ResNet152(img_channels=3, num_classes=1000):
    return ResNet(identity_block, [3, 8, 36, 3], img_channels, num_classes)

def test():
    net = ResNet50()
    x = torch.randn(2, 3, 224, 224)
    y = net(x)
    print(y.shape)
    
test()

torch.Size([2, 1000])
